In [ ]:
!nvidia-smi

Mon Apr 15 17:20:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -q accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/DataHub Projects/CrispyWork/edos/data/edos_labelled_aggregated.csv")

df.head(3)

,rewire_id,text,label_sexist,label_category,label_vector,split
0,sexism2022_english-9609,"In Nigeria, if you rape a woman, the men rape ...",not sexist,none,none,dev
1,sexism2022_english-16993,"Then, she's a keeper. 😉",not sexist,none,none,train
2,sexism2022_english-13149,This is like the Metallica video where the poo...,not sexist,none,none,train


In [ ]:
train, dev, test = df[df['split'] == 'train'], df[df['split'] == 'dev'], df[df['split'] == 'test']

print(f"train: {train.shape[0]}, dev:{dev.shape[0]}, test:{test.shape[0]}")

train: 14000, dev:2000, test:4000


In [ ]:
train

,rewire_id,text,label_sexist,label_category,label_vector,split
1,sexism2022_english-16993,"Then, she's a keeper. 😉",not sexist,none,none,train
2,sexism2022_english-13149,This is like the Metallica video where the poo...,not sexist,none,none,train
3,sexism2022_english-13021,woman?,not sexist,none,none,train
5,sexism2022_english-14998,Unlicensed day care worker reportedly tells co...,not sexist,none,none,train
6,sexism2022_english-7228,[USER] Leg day is easy. Hot girls who wear min...,sexist,3. animosity,3.3 backhanded gendered compliments,train
...,...,...,...,...,...,...
19994,sexism2022_english-12034,I blew a shot at a record like that. Girl on t...,not sexist,none,none,train
19995,sexism2022_english-5228,girls really get fucked almost every weekend ?,not sexist,none,none,train
19996,sexism2022_english-10140,The hatred for moslems are Reasonable and Just...,not sexist,none,none,train
19997,sexism2022_english-9726,Now this is a woman who gets it. 👆,not sexist,none,none,train


In [ ]:
from transformers import FalconForCausalLM, AutoTokenizer
import torch

In [ ]:
llm_path = "Rocketknight1/falcon-rw-1b"

tokenizer = AutoTokenizer.from_pretrained(llm_path, padding_side="left")

tokenizer.pad_token = tokenizer.eos_token

llm = FalconForCausalLM.from_pretrained(llm_path, load_in_8bit=True, device_map="balanced")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
def check_answer_set_tokenizer(answer) -> bool:
    return len(tokenizer(answer).input_ids) == 2

ANSWER_SET = {
    "sexist": ["yes", "sexist"],
    "not sexist": ["no", "not"],
}

index2label = {0: "sexist", 1: "not sexist"}

answer_sets_token_id = {}
for label, answer_set in ANSWER_SET.items():
    answer_sets_token_id[label] = []
    for answer in answer_set:
        print(answer,  tokenizer(answer).input_ids)
        # if check_answer_set_tokenizer(answer):
        answer_sets_token_id[label] += tokenizer(answer).input_ids[1:]


def get_probas_yes_no(outputs):
    probas_yes_no = outputs.scores[0][:, answer_sets_token_id["sexist"] + answer_sets_token_id["not sexist"]].softmax(-1)
    return probas_yes_no

def generate_for_llm(tokenized_input_data, max_new_token_length=1):
    with torch.no_grad():
        outputs = llm.generate(
            **tokenized_input_data,
            pad_token_id= tokenizer.eos_token_id,
            max_new_tokens=max_new_token_length,
            do_sample=False,
            output_scores=True,
            return_dict_in_generate=True
        )
    return outputs

def llm_predict(post, prompt_template, max_token_length=512, max_new_token_length=1, device='cpu'):

    input_data = prompt_template.replace("{POST}", post)

    tokenized_input_data = tokenizer(input_data,
                                     return_tensors="pt",
                                    #  truncation= "max_length",
                                     max_length= max_token_length)

    tokenized_input_data.to(device)

    outputs = generate_for_llm(tokenized_input_data=tokenized_input_data, max_new_token_length=max_new_token_length)
    probas_yes_no = get_probas_yes_no(outputs=outputs)
    yes_probas = probas_yes_no[:, : len(ANSWER_SET["sexist"])].sum(dim=1)
    no_proba = probas_yes_no[:, len(ANSWER_SET["not sexist"]) :].sum(dim=1)
    probas = torch.cat((yes_probas.reshape(-1, 1), no_proba.reshape(-1, 1)), -1)
    probas_per_candidate_tokens = torch.max(probas, dim=1)
    # print(probas)
    sequence_probas = [float(proba) for proba in probas_per_candidate_tokens.values]
    sequences = [index2label[int(indice)] for indice in probas_per_candidate_tokens.indices]
    return [sequences, sequence_probas]

yes [8505]
sexist [8044, 396]
no [3919]
not [1662]


In [ ]:
answer_sets_token_id

{'sexist': [396], 'not sexist': []}

In [ ]:
prompt_template = """Binary Sexism Detection: a two-class (or binary) classification where systems have to predict whether a post is sexist or not sexist.

Given a post determine whether a post is sexist or not sexist.

### Post: {POST}
### Answer: """


In [ ]:
from tqdm import tqdm
from sklearn.metrics import classification_report
import json

task_a_predict = []
task_a_labels = []
for post, label in tqdm(zip(dev['text'].tolist(), dev['label_sexist'].tolist())):
    pred, proba = llm_predict(post=post, prompt_template=prompt_template)
    task_a_predict.append(pred[0])
    task_a_labels.append(label)

result = {}
result['prompt-template'] = prompt_template
result['dev'] = classification_report(task_a_labels, task_a_predict,  output_dict=True)

path_to_save_dev = "/content/drive/My Drive/Colab Notebooks/DataHub Projects/CrispyWork/edos/LLM models/Falcon-TaskA-zeroshot-dev.json"
with open(path_to_save_dev, "w", encoding="utf-8") as outfile:
    json.dump(result, outfile, indent=4, ensure_ascii=False)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
2000it [05:02,  6.62it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetric

In [ ]:
task_a_predict = []
task_a_labels = []
for post, label in tqdm(zip(test['text'].tolist(), test['label_sexist'].tolist())):
    pred, proba = llm_predict(post=post, prompt_template=prompt_template)
    task_a_predict.append(pred[0])
    task_a_labels.append(label)

result['test'] = classification_report(task_a_labels, task_a_predict,  output_dict=True)

path_to_save_test = "/content/drive/My Drive/Colab Notebooks/DataHub Projects/CrispyWork/edos/LLM models/Falcon-TaskA-zeroshot-test.json"
with open(path_to_save_test, "w", encoding="utf-8") as outfile:
    json.dump(result, outfile, indent=4, ensure_ascii=False)

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
4000it [09:32,  6.98it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set 